In [40]:
import pandas as pd
import pymysql
import logging
import sshtunnel
from sshtunnel import SSHTunnelForwarder
import pickle
from tqdm import tqdm
from typing import List, Any, Dict
import pickle

# SSH Credentials & Tunnel

sample : https://practicaldatascience.co.uk/data-science/how-to-connect-to-mysql-via-an-ssh-tunnel-in-python

In [41]:
ssh_username = 'xiaochen'
ssh_password = '123456'
ssh_host = '10.2.56.213'
database_username = 'root'
database_password = '123456'
localhost= '127.0.0.1'
db_name = "dsta_db"



In [42]:
def open_ssh_tunnel(verbose=False):
    """Open an SSH tunnel and connect using a username and password.
    
    :param verbose: Set to True to show logging
    :return tunnel: Global SSH tunnel connection
    """
    if verbose:
        sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG
    
    global tunnel
    tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_password = ssh_password,
        remote_bind_address = ('127.0.0.1', 3306)
    )
    
    tunnel.start()




In [43]:
def close_ssh_tunnel():
    """Closes the SSH tunnel connection.
    """
    tunnel.close

# MySQL Connection
sample for insertion : https://www.w3schools.com/python/python_mysql_insert.asp

In [44]:
def mysql_connect():
    """Connect to a MySQL server using the SSH tunnel connection
    
    :return connection: Global MySQL database connection
    """
    global connection
    
    connection = pymysql.connect(
        host='localhost',
        user=database_username,
        db='dsta_db',
        passwd=database_password,
        port=tunnel.local_bind_port
    )

In [45]:
def insertion(mycursor, sql,val):
    try:
        mycursor.execute(sql, val)
        connection.commit()
    except Exception as e:
        print('\n-- DEBUG:', e)
        print('\n-- DEBUG: insertion encounter errors')
        raise
        
def update(mycursor, query):
    try:
        mycursor.execute(query)
        connection.commit()
    except Exception as e:
        print('\n-- DEBUG:', e)
        print('\n-- DEBUG: insertion encounter errors')
        raise    
    

def _selection(mycursor, sql):
    """Runs a given SQL query via the global database connection.
    
    :param sql: MySQL query
    :return: Pandas dataframe containing results
    """
    try:
        mycursor.execute(sql)

        myresult = mycursor.fetchall()
        
        return myresult
    except Exception as e:
        print('\n-- DEBUG:', e)
        print('\n-- DEBUG: selection encounter errors')
        raise

def selection(sql):
    """Runs a given SQL query via the global database connection.
    
    :param sql: MySQL query
    :return: Pandas dataframe containing results
    """
    try:
    
        return pd.read_sql_query(sql, connection)
    except Exception as e:
        print('\n-- DEBUG:', e)
        raise

In [46]:
def mysql_disconnect():
    """Closes the MySQL database connection.
    """
    
    connection.close()

## Connectivity Validity

In [47]:
open_ssh_tunnel()

mysql_connect()

mycursor = connection.cursor() 

In [48]:
print(tunnel.local_bind_port)

56482


## Insertion

In [33]:
open_ssh_tunnel()

mysql_connect()

mycursor = connection.cursor() 

In [34]:
%run "Libraries.ipynb"

In [35]:
import requests 
from bs4 import BeautifulSoup

headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }


def google(q):
    s = requests.Session()
    q = 'news:' + q
    q = '+'.join(q.split())
    url = 'https://www.google.com/search?q=' + q + '&ie=utf-8&oe=utf-8'
    r = s.get(url, headers=headers_Get)
    soup = BeautifulSoup(r.text, "html.parser")
    output = []

    for searchWrapper in soup.select('a[href*="https://"]'): #this line may change in future based on google's web page structure
        #url = searchWrapper
        url = (searchWrapper.get('href')).split('url=')[-1]
        if not ('google.com' in url or len(url) < len(q) or 'reddit.com' in url):
            #print(url)
            output.append(url)

    return output

def getNewsArticleFromURLs(title):
    from newspaper import Article
    
    urls = google(title)
    
    content = ""
    for url in urls:
        # get content 
        try:
            content = Article(url)
            content.download()
            content.parse()
            content = content.text
            if len(content.strip()) > 100:
                break
        except Exception as e:
            pass
    if content == "":
        print('\n-- DEBUG: No valid article is found for {}'.format(title))
            
    return content    

def save_pickle_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
save_pickle_object(working_df,'news_crawler/output/df/new/REDDIT_EN_1-12.pikle')

In [37]:
def is_valid_article_insertion(query):
    query_uppercase = query.upper()
    if "INSERT INTO" in query_uppercase and "VALUES" in query_uppercase:
        if "(" in query and ")" in query:
            return True
    return False

In [38]:
from dateutil import parser

def is_likely_datetime(s):
    try:
        parser.parse(s)
        return True
    except:
        return False

def is_valid_article_insertion(row):
    text = str(row['article'])
    
    lang = str(row['lang'])
    
    datetime = str(row['datetime'])
    
    if len(text) > 10 and  is_likely_datetime(datetime) and lang in ['EN', 'CN', 'BM', 'BI']:
        return True
    else:
        return False
    
def convert_article_format(article):
    if type(article) == list:
        article = article[0]

    elif type(article) == str:
        article = article

    else:
        article = ""

    return article

In [81]:
from tqdm import tqdm, trange

import ast

open_ssh_tunnel()

mysql_connect()

mycursor = connection.cursor()

def insert_comment_1df(comment_df, article_id):
    
    current_time = datetime.today().strftime("%Y-%m-%d") + 'T' + datetime.today().strftime("%H:%M")
    
    for idx, row in comment_df.iterrows():    
        
        insert_cmt_query =  "INSERT INTO dsta_db.comment (cmt_id, cmt_content, cmt_replyTo, cmt_publish_datetime, cmt_source_id, cmt_user, cmt_likes, cmt_article_id, lang, last_modified, source_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"      
        
        value = (str(row['cmt_id']), str(row['cmt_text']), str(row['cmt_replyTo']), str(row['cmt_datetime']), str(row['source']), str(row['cmt_user']), str(row['cmt_score']), str(article_id), str(row['lang']), str(current_time), str(row['source']))

        if len(row['cmt_text']) > 10:
            
            mycursor.execute(insert_cmt_query, value)
            
            connection.commit()
            
            pass
            

def insert_article_1df(news_dataset, has_comments=False):
    
    missing = 0
    
    print('\n-- DEBUG: Working on the dataset:')
    
    news_dataset = news_dataset.fillna("")

    mycursor = connection.cursor()   

    current_time = datetime.today().strftime("%Y-%m-%d") + 'T' + datetime.today().strftime("%H:%M")
    
    for idx, row in tqdm(news_dataset.iterrows(), total=news_dataset.shape[0]):
        
        insert_article_query =  "INSERT INTO dsta_db.news (category, published_datetime, org_content, org_title, source_id, url, lang, org, last_modified) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    
        #next_id = selection('SELECT article_id FROM dsta_db.news where last_modified = (select min(last_modified) from dsta_db.news);')
    
        if is_valid_article_insertion(row):
            #values.append((row['news_category'], row['datetime'], convert_article_format(row['article']), row['news_title'], row['source'], row['url'], row['lang'], row['org'], current_time))
            value = (row['news_category'], row['datetime'], convert_article_format(row['article']), row['news_title'], row['source'], row['url'], row['lang'], row['org'], current_time)
            
            mycursor.execute(insert_article_query, value)
            
            connection.commit()
            
            if has_comments:
                article_id = mycursor.lastrowid
                
                comments = row["new_comments"]
                
                try:
                    comments = ast.literal_eval(comments)
                except Exception as e:
                    pass
                    
                comments_df = pd.DataFrame.from_records(comments)                    

                comments_df['source'] = row['source']

                comments_df['lang'] = row['lang']
                
                insert_comment_1df(comments_df, article_id)
            
            #article_id = mycursor.lastrowid
    #mycursor.executemany(insert_query, values)
    #connection.commit()
    
    print('\n-- DEBUG: ',mycursor.rowcount, "was inserted.")

#df_path = "REDDIT_EN_1-12.pikle"

#working_df = pd.read_pickle("news_crawler/output/df/new/"+df_path)

#display(working_df)

directory = input("Enter dataframe directory:")
print()
df_path = input("Enter dataframe name:")
print()
try:

    working_df = pd.read_pickle(directory+df_path)
    working_df = working_df.replace(r'^\s*$', np.nan, regex=True)
    working_df = working_df.dropna(subset=['article'])
    
    print('Check Column Name Before Insertion: title, content, datetime, source_id, lang', working_df.shape)
    display(working_df.head(-5))
    
    proceed = input('Ready to Insert 0/1?')    
    if int(proceed)==1:
        print('Begin Insertion：')
        has_comments = True if int(input('Has Comment 0/1?')) == 1 else False
        insert_article_1df(working_df, has_comments=has_comments)

except Exception as e:
    print(e)
    
"news_crawler/output/df/"
"BRUDIRECT_MS_df.pickle"
"BERITA_2022-08-31_2023-12-31.pickle"
"ZB_2022-08-31_2023-12-31.pickle"
"SG_EYE_DF_1-12.pickle"

Enter dataframe directory: news_crawler/output/df/


Enter dataframe name: SG_EYE_DF_1-12.pickle



Check Column Name Before Insertion: title, content, datetime, source_id, lang (2521, 9)


,url,datetime,article,news_title,lang,org,news_category,new_comments,source
0,https://www.xinjiapoyan.com/thread-42369-1-5.html,2022-12-27 09:18:42,求助 在新加坡，把Singpass密码忘记了怎么办？ 网友们是这么回答的： 去附近的联络所，...,在新加坡，Singpass密码忘记了怎么办？,CN,1,NaN,"[{'cmt_id': '356423', 'cmt_replyTo': '', 'cmt_...",5
1,https://www.xinjiapoyan.com/thread-42368-1-6.html,2022-12-27 09:17:39,你数过每天吃的饭里都会加几勺盐吗？还在以为多吃点咸的并无大碍，但世卫组织已经要为新加坡人敲响...,饮食没“低钠”？新加坡政府：摄入盐还得摄入“严”！,CN,1,NaN,"[{'cmt_id': '356630', 'cmt_replyTo': '', 'cmt_...",5
2,https://www.xinjiapoyan.com/thread-42070-1-6.html,2022-12-20 08:54:12,在黑街街口的永福安药材店外面，墙上有个牌子写着“Bugis Street”，武疑士街（编注：...,新加坡曾有条“黑街”，时常上演生死大片...,CN,1,NaN,"[{'cmt_id': '354405', 'cmt_replyTo': '', 'cmt_...",5
3,https://www.xinjiapoyan.com/thread-42340-1-6.html,2022-12-26 21:54:57,【我爱问眼哥】 “短期来新，买哪一种预付电话卡比较划算，上网流量大的？” 新加坡重新开放，不...,“短期在新加坡，哪种电话卡上网流量多，费用最划算？”,CN,1,NaN,"[{'cmt_id': '356185', 'cmt_replyTo': '', 'cmt_...",5
4,https://www.xinjiapoyan.com/thread-42367-1-14....,2022-12-27 09:16:21,眼哥最近收到了许许多多网友的提问，大家都纷纷表示：“眼哥，飞中国流程我们都知道啦！入境新加坡...,最新入境新加坡攻略来了！来小坡前要做哪些准备？,CN,1,NaN,"[{'cmt_id': '356354', 'cmt_replyTo': '', 'cmt_...",5
...,...,...,...,...,...,...,...,...,...
2535,https://www.xinjiapoyan.com/thread-24004-1-289...,2022-1-1 16:59:41,今天中午（1月1日），人力部和贸工部发表文告宣布将工作相关活动的人数限制从50人调高至100...,新加坡进一步解封，工作相关活动开放至1000人,CN,1,NaN,"[{'cmt_id': '157353', 'cmt_replyTo': '', 'cmt_...",5
2536,https://www.xinjiapoyan.com/thread-23967-1-289...,2021-12-31 21:17:07,替14人代打冠病疫苗 印尼男子一天接种三针 为了赚钱，一名印度尼西亚男子替14人代打冠病疫苗...,印尼男子替14人代打冠病疫苗；感染奥密克戎可形成免疫,CN,1,NaN,"[{'cmt_id': '157171', 'cmt_replyTo': '', 'cmt_...",5
2537,https://www.xinjiapoyan.com/thread-23971-1-289...,2021-12-31 22:27:10,新加坡卫生部通报，12月31日新增确诊病例344起，含社区病例165起、客工宿舍病例7起，境...,12月31日新增确诊病例344起，今晚通报死亡病例1起,CN,1,NaN,"[{'cmt_id': '157125', 'cmt_replyTo': '', 'cmt_...",5
2538,https://www.xinjiapoyan.com/thread-23994-1-289...,2022-1-1 10:25:46,以下为酷航1月在中国大陆和中国澳门出发前往新加坡的航线： 【天津 - 新加坡（TR139) ...,最新！酷航发布1月份航班计划,CN,1,NaN,"[{'cmt_id': '157285', 'cmt_replyTo': '', 'cmt_...",5


Ready to Insert 0/1? 1


...

Has Comment 0/1? 1



-- DEBUG: Working on the dataset:


100%|██████████████████████████████████████████████████████████████████████████████| 2521/2521 [10:37<00:00,  3.95it/s]


-- DEBUG:  1 was inserted.


'SG_EYE_DF_1-12.pickle'

## Translate Language Database

In [89]:
open_ssh_tunnel()

mysql_connect()

mycursor = connection.cursor() 

ids = (selection('SELECT article_id FROM dsta_db.news where translated = False'))

#for idx, value in tqdm(enumerate(ids)):
#    new_value = new_ids[idx]
#    query = "UPDATE dsta_db.news_test SET news_id = '{}' WHERE news_id = '{}'".format(new_value, value)
#    update(mycursor, query)


In [32]:
max_id = selection('SELECT max(article_id) FROM dsta_db.news')
max_id = max_id.loc[0].values[0] + 1

print(max_id)

#mycursor.execute("ALTER TABLE dsta_db.news AUTO_INCREMENT = {} ".format(max_id))

mycursor.execute("ALTER TABLE dsta_db.comment AUTO_INCREMENT = {} ".format(0))

#q = "ALTER TABLE dsta_db.comment MODIFY COLUMN cmt_likes VARCHAR(10) NULL;"

#mycursor.execute("ALTER TABLE dsta_db.comment MODIFY COLUMN cmt_likes VARCHAR(100) NULL;")

320180


0

In [90]:
import requests
import json

TRANS_API_CN = 'http://10.2.56.41:8090/predictions/zh-en'

TRANS_API_BI = 'http://10.2.56.41:8082/predictions/id-en/'

TRANS_API_BM = 'http://10.2.56.41:8082/predictions/ms-en/'

# Single Sentence Translation
trans_response = requests.post(TRANS_API_CN, json={'data': '你好世界'})
#print(trans_response.text)

# Single Sentence Translation
trans_response = requests.post(TRANS_API_BI, json={'data': 'Potensi penularan COVID-19 di tengah masyaraka'})
print(trans_response.text)


Potential transmission of COVID-19 in the community


In [51]:
"2022-01-02"

is_likely_datetime("Hello")

False

In [ ]:
dir_list = os.listdir('data/news_data_output/')

In [84]:
mysql_disconnect()
close_ssh_tunnel()